In [ ]:
from torchvision.datasets import CIFAR10

## 1. Dataset

In [ ]:
train_dataset = CIFAR10(root='data', 
                        train=True, 
                        transform=None,
                        download=True)
train_dataset = CIFAR10(root='data', 
                        train=False, 
                        transform=None,
                        download=True)